In [ ]:
import boto3
import sagemaker
import pandas as pd

sess   = sagemaker.Session()
bucket = sess.default_bucket()
role = sagemaker.get_execution_role()
region = boto3.Session().region_name

sm = boto3.Session().client(service_name='sagemaker', region_name=region)

In [ ]:
!pip install -q smdebug==0.7.2
!pip install -q sagemaker-experiments==0.1.11

# Specify the S3 Location of the Features

In [ ]:
%store -r scikit_processing_job_s3_output_prefix

In [ ]:
print(scikit_processing_job_s3_output_prefix)

In [ ]:
#scikit_processing_job_s3_output_prefix = 'sagemaker-scikit-learn-2020-04-10-04-44-09-647'

print('Previous Scikit Processing Job Name: {}'.format(scikit_processing_job_s3_output_prefix))

In [ ]:
prefix_train = '{}/output/bert-train'.format(scikit_processing_job_s3_output_prefix)
prefix_validation = '{}/output/bert-validation'.format(scikit_processing_job_s3_output_prefix)
prefix_test = '{}/output/bert-test'.format(scikit_processing_job_s3_output_prefix)

train_s3_uri = 's3://{}/{}'.format(bucket, prefix_train)
validation_s3_uri = 's3://{}/{}'.format(bucket, prefix_validation)
test_s3_uri = 's3://{}/{}'.format(bucket, prefix_test)

In [ ]:
print(train_s3_uri)
!aws s3 ls $train_s3_uri/

In [ ]:
s3_input_train_data = sagemaker.s3_input(s3_data=train_s3_uri, distribution='ShardedByS3Key') 
s3_input_validation_data = sagemaker.s3_input(s3_data=validation_s3_uri, distribution='ShardedByS3Key')
s3_input_test_data = sagemaker.s3_input(s3_data=test_s3_uri, distribution='ShardedByS3Key')

print(s3_input_train_data.config)
print(s3_input_validation_data.config)
print(s3_input_test_data.config)

In [ ]:
!cat src/tf_bert_reviews.py

# Setup Hyper-Parameters

In [ ]:
epochs=1
learning_rate=0.00001
epsilon=0.00000001
train_batch_size=128
validation_batch_size=128
test_batch_size=128
train_steps_per_epoch=1000
validation_steps=1000
test_steps=1000
train_instance_count=1
train_instance_type='ml.p3.2xlarge'
train_volume_size=1024
use_xla=True
use_amp=True
max_seq_length=128
freeze_bert_layer=True
enable_sagemaker_debugger=True                    # Enable SM Debugger
#use_parameter_server=(train_instance_count >= 2) # Use Parameter Server if distributed cluster
input_mode='Pipe'                                 # 'File' or 'Pipe' Mode
run_validation=True
run_test=True
run_sample_predictions=True

# Setup Experiment

In [ ]:
import time
timestamp = '{}'.format(int(time.time()))

from smexperiments.experiment import Experiment
experiment=Experiment.create(
    experiment_name='Train-Reviews-BERT-Experiment-{}'.format(timestamp),
    description='Train Reviews BERT', 
    sagemaker_boto_client=sm)

In [ ]:
from smexperiments.tracker import Tracker
tracker_display_name='Train-Reviews-BERT-Experiment-{}'.format(timestamp)
print(tracker_display_name)

tracker = Tracker.create(display_name=tracker_display_name, sagemaker_boto_client=sm)
tracker.log_parameters({
    'epochs': epochs,
    'learning_rate': learning_rate,
    'epsilon': epsilon,
    'train_batch_size': train_batch_size,
    'validation_batch_size': validation_batch_size,
    'test_batch_size': test_batch_size,
    'train_steps_per_epoch': train_steps_per_epoch,
    'validation_steps': validation_steps,
    'test_steps': test_steps,
    'train_instance_count': train_instance_count,
    'train_instance_type': train_instance_type,
    'train_volume_size': train_volume_size,
    'use_xla': use_xla,
    'use_amp': use_amp,
    'max_seq_length': max_seq_length,
    'freeze_bert_layer': freeze_bert_layer,
    'enable_sagemaker_debugger': enable_sagemaker_debugger,
#    'use_parameter_server': use_parameter_server,
    'input_mode': input_mode, # 'File' or 'Pipe'
    'run_validation': run_validation,
    'run_test': run_test,
    'run_sample_predictions': run_sample_predictions,    
})
# we can log the s3 uri to the dataset we just uploaded
tracker.log_input(name='reviews_dataset_train', media_type='s3/uri', value=train_s3_uri)
tracker.log_input(name='reviews_dataset_validation', media_type='s3/uri', value=validation_s3_uri)
tracker.log_input(name='reviews_dataset_test', media_type='s3/uri', value=test_s3_uri)

In [ ]:
from smexperiments.trial import Trial
trial_name='Train-Reviews-BERT-Trial-{}'.format(timestamp)
trial = Trial.create(trial_name=trial_name, experiment_name=experiment.experiment_name, sagemaker_boto_client=sm)
trial.add_trial_component(tracker.trial_component)
trial_component_display_name='Train-Reviews-BERT-Trial-{}'.format(timestamp)
experiment_config={'ExperimentName': experiment.experiment_name,
                   'TrialName': trial.trial_name,
                   'TrialComponentDisplayName': trial_component_display_name}

# Setup Metrics

In [ ]:
metrics_definitions = [
     {'Name': 'train:loss', 'Regex': 'loss: ([0-9\\.]+)'},
     {'Name': 'train:accuracy', 'Regex': 'accuracy: ([0-9\\.]+)'},
     {'Name': 'validation:loss', 'Regex': 'val_loss: ([0-9\\.]+)'},
     {'Name': 'validation:accuracy', 'Regex': 'val_accuracy: ([0-9\\.]+)'},
]

# Setup Debugger
Rules and the Keras Callback Hook

In [ ]:
from sagemaker.debugger import Rule
from sagemaker.debugger import rule_configs
from sagemaker.debugger import CollectionConfig
from sagemaker.debugger import DebuggerHookConfig

rules=[
        Rule.sagemaker(
            rule_configs.loss_not_decreasing(),
            rule_parameters={
                'collection_names': 'losses,metrics',
                'use_losses_collection': 'true',
                'num_steps': '10',
                'diff_percent': '50'
            },
            collections_to_save=[
                CollectionConfig(name='losses',
                                 parameters={
                                     'save_interval': '10',
                                 }),
                CollectionConfig(name='metrics',
                                 parameters={
                                     'save_interval': '10',
                                 })
            ]
        ),
        Rule.sagemaker(
            rule_configs.overtraining(),
            rule_parameters={
                'collection_names': 'losses,metrics',
                'patience_train': '10',
                'patience_validation': '10',
                'delta': '0.5'
            },
            collections_to_save=[
                CollectionConfig(name='losses',
                                 parameters={
                                     'save_interval': '10',
                                 }),
                CollectionConfig(name='metrics',
                                 parameters={
                                     'save_interval': '10',
                                 })
            ]
        )
    ]

hook_config = DebuggerHookConfig(
    hook_parameters={
        'save_interval': '10', # number of steps
        'export_tensorboard': 'true',
        'tensorboard_dir': 'hook_tensorboard/',
    })

# Setup Tensorflow Script Mode

In [ ]:
from sagemaker.tensorflow import TensorFlow

estimator = TensorFlow(entry_point='tf_bert_reviews.py',
                       source_dir='src',
                       role=role,
                       train_instance_count=train_instance_count, # Make sure you have at least this number of input files or the ShardedByS3Key distibution strategy will fail the job due to no data available
                       train_instance_type=train_instance_type,
                       train_volume_size=train_volume_size,
                       py_version='py3',
                       framework_version='2.1.0',
                       hyperparameters={'epochs': epochs,
                                        'learning_rate': learning_rate,
                                        'epsilon': epsilon,
                                        'train_batch_size': train_batch_size,
                                        'validation_batch_size': validation_batch_size,
                                        'test_batch_size': test_batch_size,                                             
                                        'train_steps_per_epoch': train_steps_per_epoch,
                                        'validation_steps': validation_steps,
                                        'test_steps': test_steps,
                                        'use_xla': use_xla,
                                        'use_amp': use_amp,                                             
                                        'max_seq_length': max_seq_length,
                                        'freeze_bert_layer': freeze_bert_layer,
                                        'enable_sagemaker_debugger': enable_sagemaker_debugger,
                                        'run_validation': run_validation,
                                        'run_test': run_test,
                                        'run_sample_predictions': run_sample_predictions},
                       input_mode=input_mode,
                       metric_definitions=metrics_definitions,
                       rules=rules,
                       debugger_hook_config=hook_config,                       
                       train_max_run=7200 # max 2 hours * 60 minutes seconds per hour * 60 seconds per minute
                      )

# Train the Model

In [ ]:
estimator.fit(inputs={'train': s3_input_train_data, 
                      'validation': s3_input_validation_data,
                      'test': s3_input_test_data
                     },
                     experiment_config=experiment_config,                   
                     wait=False)

In [ ]:
training_job_name = estimator.latest_training_job.name
print('Training Job Name:  {}'.format(training_job_name))

In [ ]:
from IPython.core.display import display, HTML

display(HTML('<b>Review <a href="https://console.aws.amazon.com/sagemaker/home?region={}#/jobs/{}">Training Job</a> After About 5 Minutes</b>'.format(region, training_job_name)))

In [ ]:
from IPython.core.display import display, HTML

display(HTML('<b>Review <a href="https://console.aws.amazon.com/cloudwatch/home?region={}#logStream:group=/aws/sagemaker/TrainingJobs;prefix={};streamFilter=typeLogStreamPrefix">CloudWatch Logs</a> After About 5 Minutes</b>'.format(region, training_job_name)))

In [ ]:
from IPython.core.display import display, HTML

display(HTML('<b>Review <a href="https://s3.console.aws.amazon.com/s3/buckets/{}/{}/?region={}&tab=overview">S3 Output Data</a> After The Training Job Has Completed</b>'.format(bucket, training_job_name, region)))

In [ ]:
%store training_job_name

# Track Experiment

In [ ]:
from sagemaker.analytics import ExperimentAnalytics

# # Might need to convert ' => "
# search_expression = {
#     'Filters':[
#         {
#             'Name': 'DisplayName',
#             'Operator': 'Equals',
#             'Value': 'Training'
#         }
#     ]
# }

trial_component_analytics = ExperimentAnalytics(
    sagemaker_session=sess, 
    experiment_name=experiment.experiment_name,
#    search_expression=search_expression,
    sort_by='metrics.validation:accuracy.max',
    sort_order='Descending',
    metric_names=['validation:accuracy'],
    parameter_names=['epochs', 'train_batch_size']
)

In [ ]:
analytics_table = trial_component_analytics.dataframe()
analytics_table

In [ ]:
# search_expression={
#     'Filters':[{
#         'Name': 'Parents.TrialName',
#         'Operator': 'Equals',
#         'Value': ??
#     }]
# },

lineage_table = ExperimentAnalytics(
    sagemaker_session=sess,
    experiment_name=experiment.experiment_name,
#    search_expression=search_expression,
    sort_by="CreationTime",
    sort_order="Ascending",
)
lineage_table.dataframe()

# Analyze Debugger Results

## Analyze Rule Status

In [ ]:
estimator.latest_training_job.rule_job_summary()

## Analyze Tensors

Before getting to analysis, here are some notes on concepts being used in Debugger that help with analysis.
* **Trial** - object that is a center piece of Debugger API when it comes to getting access to tensors. It is a top level abstract that represents a single run of a training job. All tensors emitted by training job are associated with its trial.
* **Step** - object that represents next level of abstraction. In Debugger - step is a representation of a single batch of a training job. Each trial has multiple steps. Each tensor is associated with multiple steps - having a particular value at each of the steps.
* **Tensor** - object that represent actual tensor saved during training job. Note, a tensor can be a 1-D scaler, as well (ie. loss is stored as a scalar).

For more details on these concepts as well as on Debugger API in general (including examples) please refer to Debugger Analysis API documentation.

In [ ]:
from smdebug.trials import create_trial

# this is where we create a Trial object that allows access to saved tensors
trial = create_trial(estimator.latest_job_debugger_artifacts_path())

In [ ]:
for i in trial.tensor_names():
    print(i)

In [ ]:
import pandas as pd

def get_data(trial, tensor_name, batch_index, steps_range, mode):
    tensor = trial.tensor(tensor_name)
    vals = []
    for step_num in steps_range:
        val = tensor.value(step_num=step_num, mode=mode)[batch_index]
        vals.append(val)
    return pd.DataFrame(columns=['steps', tensor_name], data=list(zip(steps_range, vals)))

In [ ]:
from smdebug.tensorflow import modes

# Below we select the very first tensor from every batch.
# Feel free to modify this and select another tensor from the batch.
batch_index = 0

# This is a name of a tensor to analyze.
tensor_name = 'accuracy'

steps = 0
while steps == 0:
    # trial.steps return all steps that have been downloaded by Debugger to date.
    # It doesn't represent all steps that are to be available once training job is complete -
    # it is a snapshot of a current state of the training job. If you call it after training job is done
    # you will get all tensors available at once.
    steps = trial.steps()
    print('Waiting for tensors to become available...')
    time.sleep(3)
print('\nDone')

print('Getting tensors...')
rendered_steps = []

# trial.loaded_all_steps is a way to keep monitoring for a state of a training job as seen by Debugger.
# When SageMaker completes training job Debugger, and trial, becomes aware of it.

loaded_all_steps = False
while not loaded_all_steps:
    loaded_all_steps = trial.loaded_all_steps
    steps = trial.steps()
    # show diff between lists
    steps_to_render = list(set(steps).symmetric_difference(set(rendered_steps)))
    
    data = get_data(trial=trial, 
                    tensor_name=tensor_name, 
                    batch_index=0, 
                    steps_range=steps_to_render, 
                    mode=modes.GLOBAL)
    print(data)
    data.plot(x='steps', y=tensor_name)
    
    rendered_steps.extend(steps_to_render)
    time.sleep(5)

## Visualize Loss

In [ ]:
from smdebug.tensorflow import modes

import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format='retina'

#from matplotlib import pyplot as plt

import pandas as pd

# Let's visualize weights of the first convolutional layer as they progressively change through training.
tensor_name = 'loss'

num_batches = trial.tensor(tensor_name).value(step_num=steps[0]).shape[0]
for batch_index in range(0, num_batches):
    steps_range = trial.tensor(tensor_name).steps()
    print(steps_range)
    data = get_data(trial=trial, 
                    tensor_name=tensor_name, 
                    batch_index=batch_index, 
                    steps_range=steps_range, 
                    mode=modes.GLOBAL)
    print(data)
    data.plot(x='steps', y=tensor_name)

## Visualize Accuracy

In [ ]:
from smdebug.tensorflow import modes

import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format='retina'

#from matplotlib import pyplot as plt

import pandas as pd

tensor_name = 'accuracy'

num_batches = trial.tensor(tensor_name).value(step_num=steps[0]).shape[0]
for batch_index in range(0, num_batches):
    steps_range = trial.tensor(tensor_name).steps()
    print(steps_range)
    data = get_data(trial=trial, 
                    tensor_name=tensor_name, 
                    batch_index=batch_index, 
                    steps_range=steps_range, 
                    mode=modes.GLOBAL)
    print(data)
    data.plot(x='steps', y=tensor_name)

## Add Lambda Function to Stop the TrainingJob Early
In your AWS console, go to Lambda Management Console,
Create a new function by hitting Create Function,
Choose the language as Python 3.7 and put in the following sample code for stopping the training job if one of the Rule statuses is "IssuesFound".

### CloudWatch Events for Rules
Rule status changes in a training job trigger CloudWatch Events. These events can be acted upon by configuring a CloudWatch Rule (different from Amazon SageMaker Debugger Rule) to trigger each time a Debugger Rule changes status. In this notebook we'll go through how you can create a CloudWatch Rule to direct Training Job State change events to a lambda function that stops the training job in case a rule triggers and has status "IssuesFound".

Create a new execution role for the Lambda, and
In your IAM console, search for the role and attach "AmazonSageMakerFullAccess" policy to the role. This is needed for the code in your Lambda function to stop the training job.


### Create a CloudWatch Rule to Trigger a Lamba
In your AWS Console, go to CloudWatch and select Rule from the left column,
Hit Create Rule. The console will redirect you to the Rule creation page,
For the Service Name, select "SageMaker".
For the Event Type, select "SageMaker Training Job State Change".
In the Targets select the Lambda function you created above, and
For this example notebook, we'll leave everything as is.


#### Create the Lambda

```
import json
import boto3
import logging

def lambda_handler(event, context):
    training_job_name = event.get("detail").get("TrainingJobName")
    eval_statuses = event.get("detail").get("DebugRuleEvaluationStatuses", None)

    if eval_statuses is None or len(eval_statuses) == 0:
        logging.info("Couldn't find any debug rule statuses, skipping...")
        return {
            'statusCode': 200,
            'body': json.dumps('Nothing to do')
        }

    client = boto3.client('sagemaker')

    for status in eval_statuses:
        if status.get("RuleEvaluationStatus") == "IssuesFound":
            logging.info(
                'Evaluation of rule configuration {} resulted in "IssuesFound". '
                'Attempting to stop training job {}'.format(
                    status.get("RuleConfigurationName"), training_job_name
                )
            )
            try:
                client.stop_training_job(
                    TrainingJobName=training_job_name
                )
            except Exception as e:
                logging.error(
                    "Encountered error while trying to "
                    "stop training job {}: {}".format(
                        training_job_name, str(e)
                    )
                )
                raise e
    return None
```

* Create a new execution role for the Lambda, and
* In your IAM console, search for the role and attach "AmazonSageMakerFullAccess" policy to the role. This is needed for the code in your Lambda function to stop the training job.

#### Create a CloudWatch Rule

* In your AWS Console, go to CloudWatch and select Rule from the left column,
* Hit Create Rule. The console will redirect you to the Rule creation page,
 * For the Service Name, select "SageMaker".
 * For the Event Type, select "SageMaker Training Job State Change".
* In the Targets select the Lambda function you created above, and
* For this example notebook, we'll leave everything as is.

SageMaker kicked off rule evaluation jobs, one for each of the SageMaker rules - `Overtraining` and `LossNotDecreasing` as specified in the estimator. If we setup a CloudWatch Rule to stop the training job, we would see the `TrainingJobStatus` change to `Stopped` once the `RuleEvaluationStatus` for changes to `IssuesFound`.

In [ ]:
# # This utility gives the link to monitor the CW event
# def _get_rule_job_name(training_job_name, rule_configuration_name, rule_job_arn):
#         """Helper function to get the rule job name"""
#         return "{}-{}-{}".format(
#             training_job_name[:26], rule_configuration_name[:26], rule_job_arn[-8:]
#         )
    
# def _get_cw_url_for_rule_job(rule_job_name, region):
#     return "https://{}.console.aws.amazon.com/cloudwatch/home?region={}#logStream:group=/aws/sagemaker/ProcessingJobs;prefix={};streamFilter=typeLogStreamPrefix".format(region, region, rule_job_name)


# def get_rule_jobs_cw_urls(estimator):
#     region = boto3.Session().region_name
#     training_job = estimator.latest_training_job
#     training_job_name = training_job.describe()["TrainingJobName"]
#     rule_eval_statuses = training_job.describe()["DebugRuleEvaluationStatuses"]
    
#     result={}
#     for status in rule_eval_statuses:
#         if status.get("RuleEvaluationJobArn", None) is not None:
#             rule_job_name = _get_rule_job_name(training_job_name, status["RuleConfigurationName"], status["RuleEvaluationJobArn"])
#             result[status["RuleConfigurationName"]] = _get_cw_url_for_rule_job(rule_job_name, region)
#     return result

# get_rule_jobs_cw_urls(estimator)